# Get Camera Metadata (Firemap API)

<b>Summary:</b><br>
Get all camera metadata available from firemap pylaski api and save as csv.

<b>Output:</b><br>
.<br>
└── data<br>
&emsp;&emsp;&emsp;└── raw<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── camera_metadata.csv

<b>Areas for Improvement:</b><br>
- Cleanup data by incorporating manual metadata updates made at source.

## Get All Camera Metadata From Firemap API

In [1]:
import json
from datetime import datetime

import pandas as pd
import pytz
import requests

In [2]:
BASE_URL = "https://firemap.sdsc.edu/pylaski/"

In [3]:
# Get data for ALL cameras
# https://firemap.sdsc.edu/pylaski/stations?camera=only&selection=boundingBox&minLat=0&maxLat=90&minLon=-180&maxLon=0
entity = "stations"
payload = {
    "camera": "only",
    "selection": "boundingBox",
    "minLat": 0,
    "maxLat": 90,  # northern hemisphere
    "minLon": -180,
    "maxLon": 0,  # western hemisphere
}
response = requests.get(f"{BASE_URL}{entity}", params=payload)

In [4]:
if response.status_code in range(200, 300):  # successful response range
    response_json = response.json()
    features = response_json["features"]
else:
    # TODO:
    # Add error handling
    print("Error!")

In [5]:
cameras_df = pd.json_normalize(features)
print(f"Total cameras: {len(cameras_df['properties.description.id'])}")

Total cameras: 1158


In [6]:
cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.is_patrol_mode,properties.description.lastupdate,properties.description.region,properties.description.fov,properties.description.activated_at,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current
0,Feature,Point,"[-117.12, 32.55, 10]",,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Camera breakdown
# all camera metadata either has hpwren or axis prefix
cameras_df["properties.description.id"].str.split(r"_|-", expand=True)[0].str.extract(
    r"([a-zA-Z]+)"
)[0].str.lower().value_counts()

axis      1055
hpwren     103
Name: 0, dtype: int64

## Write camera metadata to raw folder

In [8]:
# current_time_str = datetime.now(tz=pytz.UTC).strftime("%Y%m%d_%H_%M_%S")

cameras_df.to_csv(f"../../data/raw/camera_metadata.csv", index=False)